# **Xây dựng mô hình LivenessNet**
<br><br>

## **Kiến trúc mô hình**

### **1. Bài toán**

Bài toán **LivenessNet** yêu cầu phân biệt 1 frame ảnh trước camera là mặt thật hay là 1 kiểu proofing 2d bằng ảnh in, màn hình điện thoại hay màn hình máy tính

**Yêu cầu:**
* Học được các viền ảnh
* Học được phản chiếu ánh sáng lên trên điện thoại khi proofing bằng điện thoại
* Học được texture giấy( khi proofing bằng ảnh in )...
* Nhẹ và chạy được theo thời gian thực
* Thời gian huấn luyện phù hợp với tài nguyên CPU

**Input:** Ảnh đã được resize về kích thước **64 x 64** ( sẽ tăng kích thước lên thành **128 x 128** nếu **64 x 64** không đáp ứng được yêu cầu)

**Output:** một giá trị xác suất trong khoảng **[0,1]** là xác suất ảnh là thật

### **2. Kiến trúc mô hình**

**Kiến trúc sử dụng:** Vì cần học các đặc trưng đơn giản như viền cạnh, ánh sáng phản chiếu nên ta sẽ sử dụng một mạng **CNN** đơn giản. Mạng **CNN** đơn giản đủ để phát hiện các đặc trưng thô và tinh cơ bản. Mạng được xây dựng theo kiến trúc sau:
* **Lớp 1: Conv2D**
    * Số filter: 32
    * Kích thước filter: 3x3
    * Padding cho kích thước giữ nguyên với kích thước đầu vào
    * Hàm kích hoạt : ReLU
    * Đầu vào: 64x64x3
    * Đầu ra: 64x64x32

* **Lớp 2: BatchNormalization**
    * Chuẩn hóa của lớp tích chập 2D phía trước

* **Lớp 3: MaxPooling2D**
    * Kích thước cửa sổ: 2x2
    * Đầu vào: 64x64x32
    * Đầu ra: 32x32x32

* **Lớp 4: Conv2D**
    * Số filter: 64 ( tăng số lượng filter để học được nhiều đặc trưng hơn)
    * Kích thước filter: 3x3
    * Padding: "same" ( cho giống với kích thước đầu vào)
    * Hàm kích hoạt: ReLU
    * Đầu vào: 32x32x32
    * Đầu ra: 32x32x64

* **Lớp 5: BatchNormalization**
    * Chuẩn hóa của lớp tích chập 2D phía trước

* **Lớp 6: MaxPooling2D**
    * Kích thước cửa sổ: 2x2
    * Đầu vào: 32x32x64
    * Đầu ra: 16x16x64

* **Lớp 7: Conv2D**
    * Số filter: 128 ( tăng số lượng filter để học được nhiều đặc trưng hơn)
    * Kích thước filter: 3x3
    * Padding: "same" ( cho giống với kích thước đầu vào)
    * Hàm kích hoạt: ReLU
    * Đầu vào: 16x16x64
    * Đầu ra: 16x16x128

* **Lớp 8: BatchNormalization**
    * Chuẩn hóa của lớp tích chập 2D phía trước

* **Lớp 9: MaxPooling2D**
    * Kích thước cửa sổ: 2x2
    * Đầu vào: 16x16x128
    * Đầu ra: 8x8x128

* **Lớp 10: Conv2D**
    * Số filter: 128 ( tăng số lượng filter để học được nhiều đặc trưng hơn)
    * Kích thước filter: 3x3
    * Padding: "same" ( cho giống với kích thước đầu vào)
    * Hàm kích hoạt: ReLU
    * Đầu vào: 8x8x128
    * Đầu ra: 8x8x128

* **Lớp 11: BatchNormalization**
    * Chuẩn hóa của lớp tích chập 2D phía trước

* **Lớp 12: MaxPooling2D**
    * Kích thước cửa sổ: 2x2
    * Đầu vào: 8x8x128
    * Đầu ra: 4x4x128

* **Lớp 13: Flatten**
    * Chuyển ma trận 4x4x128 về vector 1 chiều
    * Đầu ra: 4x4x128 = 2048 chiều

* **Lớp 14: Fully Connected**
    * Số neuron: 256
    * Hàm kích hoạt: ReLU
    * Đầu vào: 2048
    * Đầu ra: 256

* **Lớp 15: Dropout**
    * Tỉ lệ dropout: 0.5( tắt ngẫu nhiên 50% neuron trong quá trình huấn luyện)
    * Đầu ra 256

* **Lớp 16: Fully Connected( Lớp đầu ra)**
    * Số neuron: 1
    * Hàm kích hoạt: Sigmoid
    * Đầu vào: 256
    * Đầu ra: 1( xác suất ảnh lả real/ fake)

### **3. Xây dựng mô hình**

In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
# Import thư viện
import torch
import torch.nn as nn

# Định nghĩa lớp mô hình, lớp mô hình phải được kế thừa từ lớp cha nn.Module
class LivenessNet( nn.Module ):
    
    # Xây dựng phương thức khởi tạo
    # Trong phương thức khởi tạo thì xây dựng kiến trúc mạng
    def __init__(self):
        # Gọi phương thức khởi tạo của lớp cha
        super(LivenessNet, self).__init__()

        # Xây dựng kiến trúc mạng
        self.network = nn.Sequential(
            # Đầu vào 64x64x3
            nn.Conv2d(
                in_channels=3, 
                out_channels=32, 
                kernel_size=3,
                padding="same"
                ),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            # đầu ra 64x64x32
            nn.MaxPool2d(
                kernel_size=2,
                stride=2,
            ),
            # đầu ra là 32x32x32
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                padding="same"
            ),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            # đầu ra là 32x32x64
            nn.MaxPool2d(
                kernel_size=2,
                stride=2
            ),
            # đầu ra là 16x16x64
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                padding="same"
            ),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            # đầu ra 16x16x128
            nn.MaxPool2d(
                kernel_size=2,
                stride=2
            ),
            #đầu ra 8x8x128
            nn.Conv2d(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
                padding="same"
            ),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            #đầu ra 8x8x128
            nn.MaxPool2d(
                kernel_size=2,
                stride=2
            ),
            # đầu ra 4x4x128
            nn.Flatten(),
            nn.Linear(
                in_features=2048,
                out_features=256,
            ),
            # đầu ra 256
            nn.Dropout(),
            nn.Linear(
                in_features=256,
                out_features=2
            ),
            nn.Sigmoid()   
        )

    def forward(self, x):
        return self.network(x)

## **Huấn luyện mô hình**

In [3]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### **Nạp dữ liệu**

In [23]:
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import os

# Đường dẫn đến thư mục gốc của thư mục chứa data
data_path = os.path.join("..","Data","Dataset")
# Định nghĩa hàm resize ảnh
transformImage = transforms.Compose(
    [transforms.Resize((64, 64)), transforms.ToTensor()]
    )
# Load data 
dataset = ImageFolder(root = data_path, transform=transformImage)

In [24]:
len(dataset)

6894

### **Chia tập train, test**

In [9]:
%pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset

# Tạo danh sách các chỉ số của tập dataset gốc
indices = list(range(len(dataset)))

# Lấy danh sách các nhãn của tập dataset
labels = [dataset[i][1] for i in indices]

# Chia tập train, test, tỉ lệ các mẫu trong mỗi tập train, test sẽ tương tự nhau
train_idx, test_idx = train_test_split(indices,test_size= 0.3, random_state = 42, stratify= labels )

# Tạo các đối tượng Subset từ tập chỉ số train, và test vừa lấy được
trainDataset = Subset(dataset, train_idx)
testDataset = Subset(dataset, test_idx)

print("train: ", len(trainDataset))
print("test: ", len(testDataset))

train:  4825
test:  2069


### **Tạo batch các dữ liệu**

In [26]:
from torch.utils.data import DataLoader

dataLoader = DataLoader(trainDataset, 16, False)


### **Huấn luyện**

#### **Chọn các thành phần huấn luyện**

* Khởi tạo mô hình

* Hàm loss: *Cross Entropy* ( phù hợp với phân loại nhị phân)

* Chiến lược cập nhật trọng số: *Adam* ( phổ biến)
    * `lr`: tốc độ học : *0.001*

* Số epochs huấn luyện: 100

In [46]:
import torch.optim as optimize

model = LivenessNet()
loss_function = torch.nn.CrossEntropyLoss()
epochs = 10
lr = 0.001
optimizer = optimize.Adam(model.parameters(), lr)

### **Định nghĩa hàm train**

In [ ]:
def train(model, loss_function, epochs, optimizer, dataloader):
    model.train()
    for i in range(epochs):
        print("Epoch: ", i)
        for image, label in dataloader:
            # Feedforward
            output = model(image)
            # Tính toán hàm mất mát
            loss = loss_function(output, label)
            # Lan truyền gradient
            # Xóa gradient về 0
            optimizer.zero_grad()
            # Lan truyền gradient
            loss.backward()
            # Cập nhật trọng số
            optimizer.step()

### **Thực hiện huấn luyện**

In [47]:
train(model, loss_function, epochs, optimizer, dataLoader)

Epoch:  0
Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9


## **Lưu mô hình**

In [48]:
path = "checkpoint.pth"
torch.save(
    model.state_dict(),
    path
)
print("Lưu thành công")

Lưu thành công


## **Test**

In [125]:
def predict(model,image):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    model.to(device)
    image.to(device)
    with torch.no_grad():
        output = model(image)
        predict = torch.argmax(output, dim = 1)
    return predict

In [142]:
def accuracy(model, testDataset):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    dataLoader = DataLoader(testDataset, batch_size=64, shuffle = False)
    
    trueLabel = 0
    totalSample = len(testDataset)

    with torch.no_grad():
        for image, label in dataLoader:
            # Di chuyển các mẫu lên thiết bị tính toán( là GPU nếu GPU sẵn sàng, không thì là CPU)
            image = image.to(device)
            label = label.to(device)

            predictOutput = predict(model, image)

            number_true_label = torch.sum(predictOutput == label)
            trueLabel +=  number_true_label
    return (trueLabel/totalSample).item()
accuracy(model,testDataset)

0.9647172689437866